# Final Location Finder
After over 200 hours of training (compared to the prelims 4 hours), the final model is ready for prime time! While still not perfect, it should generate more accurate results for the mid to large organs.

An example of this technology is the nnU-Net framework (linke to their nature publication: https://www.nature.com/articles/s41592-020-01008-z), which is a deep learning model for medical image segmentation. This model can be used to locate and idenpngy different organs in medical images in 3D space. In this notebook, we will demonstrate how to use a nnU-Net model trained across 120 abdomen CT and 40 MRI scans to locate and idenpngy structures in your image stack from either image technology.

The output of the model will be a prediction on the location of the structures in the image stack. The prediction will be in the form of a description that describes the location of the structures in the image stack. The prediction will be displayed in the notebook.

## There is no need to upload anything! 
Please run each cell in order and select your group from the drop down to run all your files at once!

In [ ]:
# Final Location finder
!pip install ipywidgets nnUNetv2 requests


In [ ]:
import os
import nibabel as nib
import numpy as np
import tifffile
import shutil
import ipywidgets as widgets
from IPython.display import display
import json
# Define the root directory for all groups and local storage for results
root_directory = "/project/cec/class/bme301_sp25/Compressed_groups_final/"
notebook_results_folder = "./processed_results"
os.makedirs(notebook_results_folder, exist_ok=True)
extracted_path = "/project/cec/class/bme301_sp25/Results_Folder/Results_Folder/"
os.environ['nnUNet_raw'] = extracted_path
os.environ['nnUNet_results'] = extracted_path
os.environ['nnUNet_preprocessed'] = extracted_path

def list_group_folders():
    """List available group folders."""
    return [folder for folder in os.listdir(root_directory) if folder.startswith("group_")]

# Create dropdown for group selection
group_dropdown = widgets.Dropdown(
    options=list_group_folders(),
    description='Group:',
    layout=widgets.Layout(width='400px')
)

output = widgets.Output()

def find_nnunet_predict():
    home_dir = os.path.expanduser("~")
    local_bin_dir = os.path.join(home_dir, ".local", "bin")
    nnunet_path = shutil.which("nnUNetv2_predict", path=local_bin_dir)
    if nnunet_path is None:
        raise FileNotFoundError("nnUNetv2_predict not found. Ensure it is installed and accessible.")
    return nnunet_path
  
def run_nnunet_prediction(selected_group):
    """Runs nnUNetv2 prediction on all preprocessed NIfTI files in the selected group and stores results locally."""
    nnunet_path = find_nnunet_predict()
    if nnunet_path is None:
        raise FileNotFoundError("nnUNetv2_predict not found. Ensure it is installed and accessible.")
    
    group_path = os.path.join(root_directory, selected_group)
    preprocessed_folder = os.path.join(group_path, "preprocessed_nifti")
    result_folder = os.path.join(notebook_results_folder, selected_group)
    os.makedirs(result_folder, exist_ok=True)
    
    if not os.path.exists(preprocessed_folder) or not os.listdir(preprocessed_folder):
        print(f"Error: No preprocessed NIfTI files found for {selected_group}. Skipping processing.")
        return
    
    print(f"Processing only the selected group: {selected_group}")
    subfolders = [f for f in os.listdir(preprocessed_folder) if os.path.isdir(os.path.join(preprocessed_folder, f))]
    for subfolder in subfolders:
        input_folder = os.path.join(preprocessed_folder, subfolder)
        input_file = os.path.join(input_folder, "amos_0001_0000.nii.gz")
        
        if not os.path.exists(input_file):
            print(f"Error: NIfTI file missing in {input_folder}, skipping {subfolder}.")
            continue
        
        output_subfolder = os.path.join(result_folder, subfolder)
        os.makedirs(output_subfolder, exist_ok=True)
        
        print(f"Running nnUNetv2_predict on {input_file}...")
        command = f"{nnunet_path} -i {input_folder} -o {output_subfolder} -d 123 -c 3d_fullres"
        os.system(command)
        print(f"nnUNet prediction completed for {subfolder}.")
    
    analyze_output(result_folder)

def analyze_output(selected_group):
    """Analyze the segmentation output from nnUNet for each processed file in all subfolders of the selected group."""
    
    # Define the root results folder for the selected group
    group_output_folder = selected_group
    
    if not os.path.exists(group_output_folder):
        raise FileNotFoundError(f"Output folder not found for group {selected_group}: {group_output_folder}")
    
    # Path to plan2.json for mapping region numbers
    plan2_path = os.path.join(os.environ['nnUNet_raw'], "Dataset123_Foo", "nnUNetTrainer__nnUNetPlans__3d_fullres", "plan2.json")
    if not os.path.exists(plan2_path):
        raise FileNotFoundError(f"plan2.json not found in {plan2_path}.")

    with open(plan2_path, 'r') as f:
        plan2_data = json.load(f)

    # Search each subfolder inside the group output folder
    for subfolder in os.listdir(group_output_folder):
        subfolder_path = os.path.join(group_output_folder, subfolder)
        
        if not os.path.isdir(subfolder_path):  
            continue  # Skip non-folder files
        
        nii_files = [f for f in os.listdir(subfolder_path) if f.endswith('.nii.gz')]

        if not nii_files:
            continue

        for nii_file in nii_files:
            file_path = os.path.join(subfolder_path, nii_file)

            img = nib.load(file_path)
            image_data = img.get_fdata()
            unique_regions = np.unique(image_data)

            # Remove 0 (background)
            filtered_regions = unique_regions[unique_regions > 0]

            # Translate numbers into region names
            region_names = [plan2_data.get(str(int(region)), "Unknown") for region in filtered_regions]

            # Print the subfolder name along with the extracted regions
            print(f"{subfolder}: {', '.join(region_names) if region_names else 'Unknown'}")


def process_selected_group(_):
    selected_group = group_dropdown.value
    with output:
        output.clear_output()
        print(f"Processing {selected_group}...")
        run_nnunet_prediction(selected_group)

# Button to process selected group
process_button = widgets.Button(
    description='Process Group',
    layout=widgets.Layout(width='200px', height='40px')
)
process_button.on_click(process_selected_group)

display(widgets.VBox([group_dropdown, process_button, output]))


# Mannual Debug 

In [ ]:
import os
import nibabel as nib
import numpy as np
import json
import ipywidgets as widgets
from IPython.display import display

def list_group_folders():
    """List available group folders inside ./processed_results/."""
    results_root = "./processed_results"
    if not os.path.exists(results_root):
        return []
    return [folder for folder in os.listdir(results_root) if folder.startswith("group_")]

def analyze_output(selected_group):
    
    # Define the root results folder for the selected group
    group_output_folder = os.path.join("./processed_results", selected_group)
    
    if not os.path.exists(group_output_folder):
        print(f"Output folder not found for group {selected_group}: {group_output_folder}")
        return
    
    # Path to plan2.json for mapping region numbers
    plan2_path = os.path.join(os.environ.get('nnUNet_raw', ""), "Dataset123_Foo", "nnUNetTrainer__nnUNetPlans__3d_fullres", "plan2.json")
    if not os.path.exists(plan2_path):
        print(f"plan2.json not found in {plan2_path}.")
        return

    with open(plan2_path, 'r') as f:
        plan2_data = json.load(f)

    results = {}

    # Search each subfolder inside the group output folder
    for subfolder in os.listdir(group_output_folder):
        subfolder_path = os.path.join(group_output_folder, subfolder)
        
        if not os.path.isdir(subfolder_path):  
            continue  # Skip non-folder files
        
        nii_files = [f for f in os.listdir(subfolder_path) if f.endswith('.nii.gz')]

        if not nii_files:
            continue

        for nii_file in nii_files:
            file_path = os.path.join(subfolder_path, nii_file)

            img = nib.load(file_path)
            image_data = img.get_fdata()
            unique_regions = np.unique(image_data)

            # Remove 0 (background)
            filtered_regions = unique_regions[unique_regions > 0]

            # Translate numbers into region names
            region_names = [plan2_data.get(str(int(region)), "Unknown") for region in filtered_regions]

            # Store the results
            results[subfolder] = region_names if region_names else ["Unknown"]

    # Display results
    print(f"\nFinal Results for Group {selected_group}:")
    for subfolder, regions in results.items():
        print(f"{subfolder}: {', '.join(regions)}")

# Create dropdown for group selection
group_dropdown = widgets.Dropdown(
    options=list_group_folders(),
    description='Select Group:',
    layout=widgets.Layout(width='300px')
)

analyze_button = widgets.Button(
    description="Analyze Group",
    button_style='primary'
)

output = widgets.Output()

def on_analyze_button_click(_):
    with output:
        output.clear_output()
        analyze_output(group_dropdown.value)

analyze_button.on_click(on_analyze_button_click)

# Display UI
display(widgets.VBox([group_dropdown, analyze_button, output]))
